<a href="https://colab.research.google.com/github/rrajasek95/nlp-243-notebooks/blob/main/Section_2_Python_Concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP 243 Section 02 - Python Concepts

In this notebook, we explore will be exploring intermediate concepts in Python that will become more relevant as we progress in the course. 

The outline of the section is as follows:
* Python Classes
* File I/O
* Data Serialization
* Object Serialization

# New Section

## Python Classes

Python supports Object Oriented Programming through the concept of classes. To give a refresher, classes are programming constructs that provide a template/blueprint to group information and behavior acting on that information into a single unit. 

This unit is what's referred to as an *object* or an *instance*. All values in Python are defined as objects. 

Examples of classes include the `int`, `str`, `float` classes. Examples of objects of those classes include `1`, `"hello"` and `1.0`. 

Thus, in this case, we can view classes as types for data. One useful aspect is that we can define more complex types and program units using Object Oriented Programming. 

### Case Study 1: Defining a min-heap using classes

One powerful use case for Object Oriented Programming is to define custom data structures. Python has a [heapq](https://docs.python.org/3/library/heapq.html) module which uses functions to modify an array to use it as a priority queue.

It defines the following methods:

* `heapq.heappush(heap, item)` that pushes `item` into the array `heap`
* `heapq.heappop(heap)` that pops and returns the smallest item from the heap.
* `heapq.heappushpop(heap, item)` that pushes `item` and pops the smallest item off the heap. This is more efficient than calling `heappush` followed by `heappop`.

The issue with this sort of implementation is that you have to annoyingly pass the same list as an argument over and over again to use it in your program. We could instead wrap this implementation into a class.

In [ ]:
import heapq

class Heap(object):
    """
    Wrapper class for some of the heapq methods.
    """
    def __init__(self): 
        # Constructor:
        # Initializes the underlying heap array
        self.heapArray = []

    def push(self, item):
        heapq.heappush(self.heapArray, item)
    
    def pop(self):
        return heapq.heappop(self.heapArray)
    
    def pushpop(self, item):
        return heapq.heappushpop(self.heapArray, item)

    def __len__(self):
        return len(self.heapArray)

This allows us to neatly use heaps as priority queues.

In [ ]:
hq = Heap()

hq.push((2, 'Test'))
hq.push((1, 'Hello'))
hq.push((3, 'World'))

while len(hq) > 0:
    print(hq.pop())

(1, 'Hello')
(2, 'Test')
(3, 'World')


### Case Study 2: Extending heap for modifying behavior

The heap class we defined behaves as a min-heap. However, this may not be useful for our purposes especially when we need to use them as a max-heap. Furhtermore, the `heapq` module does not provide a proper implementation of a max-heap. 

We will extend the heap class we supported to provide the max-heap behavior. One insight that will help us to implement it is how a min-heap is implemented. 

A min-heap places the smallest element at the root of the heap. It does so by comparing itself with some of the other elements such that it is less than them. Therefore, if we want max-heap behavior, we need to convert that "less-than" comparison to a "greater-than" comparison. 

In [ ]:
"""
Solution derived from the following StackOverflow answer:
https://stackoverflow.com/a/40455775

This is merely an illustrative example to show how subclassing works. 
This may not be representative of best practice for implementing these classes.
"""

class MaxHeapItem(object):
    # The MaxHeapItem inverts the comparison operator 
    # to convert a less-than comparison to a greater-than comparison
    # this is a hack to force the min-heap to behave as a max-heap.
    
    def __init__(self, val):
        self.val = val

    def __lt__(self, other): # if x < y, place x before y; if x > y, we place x before y
        # Note: normal implementation of less than would be
        # self.val < other.val ; This inverts the comparison operator
        return self.val > other.val

    def __eq__(self, other):
        return self.val == other.val

    def __str__(self): 
        return str(self.val)
    
# Normal python implementation tries to place the smallest value at the top
# What we want: is to place the largest value on top
# compare(x, y): if x < y, it places x above y in the heap
# What we want is compare(x, y): if x > y, we place x above y in the heap

class MaxHeap(Heap): 
    # Placing Heap between parentheses indicates we are subclassing it
    # i.e. we will inherit all the underlying methods of the heap

    def push(self, item): 
        # We retain the underlying min heap behavior
        # by calling super().push etc...
        super().push(MaxHeapItem(item))
    
    def pop(self):
        return super().pop().val
    
    def pushpop(self, item):
        return super().pushpop(MaxHeapItem(item))

hq2 = MaxHeap()

hq2.push((2, 'Test'))
hq2.push((1, 'Hello'))
hq2.push((3, 'World'))

while len(hq2) > 0:
    print(hq2.pop())

(3, 'World')
(2, 'Test')
(1, 'Hello')


## Python File I/O

Throughout this course, you will rely on reading and writing to files and storing data in various formats.

This section will cover some specific ways to perform file I/O, as well as some custom formats such as JSON, CSV.

#### Vanilla File I/O

File I/O typically requires us to make an Operation System call (or syscall for short) to *open* a file for a certain action. 

There are three common actions (or modes) supported:
* Read
* Write (Starting from the beginning of the file)
* Append (Write starting at the end of the file)

You can open a file with one or more modes if necessary, but normally it's preferable to open a file with only one or the other mode to ensure you don't perform unintended actions such as writing to a file you only wanted to read.

Once you're done reading or writing to a file, you will have to *close* the file. The reason is that the OS maintains information about open files in memory. Failing to close them can lead to that memory not being freed - causing a memory leak.

##### Manual File I/O

A typical file I/O workflow that's commonly done is the following:
* Open the file
* Read/ Write/ Append to the file
* Close the file

The above is what we will be doing in this course.

Other workflows include:
* Open the file and keep writing to it (logger workflow)
* Open the file and keep reading from it (streaming file workflow)

The above cases occur with daemon processes or long-running programs, therefore, when those programs are terminated, we will need to close the files appropriately as a clean-up step.

In [ ]:
# Please refer to the documentation for further info:
# https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files

########################################################
# Opening a file in python and writing some data to it #
########################################################
test_file = open('file.txt', 'w') # w indicates that only writing can be done

# Using the flag 'w' allows us to write the data as a string
test_file.write('Hello, World!')

# Add more data
test_file.write(' Additional Data')
test_file.writelines(['\nData on a new line\n', 'Some more data'])
test_file.close() # Close the file to free memory

############################################
# Now opening the same file and reading it #
############################################
same_file = open('file.txt', 'r') # r indicates that only reading can be done

# Calling read without arguments will read the entire file
content = same_file.readlines()

same_file.close()

print(content)

['Hello, World! Additional Data\n', 'Data on a new line\n', 'Some more data']


##### Context Managers

In the above section, we talked about the first use case and the need for closing files. 

One particular concern that arises is what happens during failure.

In our workflow:
* Open the file
* Read/ Write/ Append to the file
* Close the file

We have three failure cases:
* We fail to open the file
* We fail to read / write / append to the file
* We fail to close the file

This leads to the following handling cases to ensure that we have no open files:
* File is not open, so we must do nothing
* We failed to read/write, so we may have to close the file
* We will have to try closing it, otherwise give up.

In [ ]:
print("Write failure case:\n---")
try:
    file2 = open('file2.txt', 'w')
    # We are trying to write an integer when write expects a string
    file2.write(3)
    file2.close()
except TypeError:
    # Program throws an error
    print(file2.closed) # file2 remains open, so we must close it
    file2.close()

print("\nFile not found case:\n---")
try:
    file3 = open('file3.txt', 'r') # We are trying to read a nonexistent file
    data = file3.read()
    file3.close()
except FileNotFoundError:
    print("File not found") # File is not found, so we should do nothing
except IOError: # file is found but throws an error, so close in this case
    file3.close()

print("\nAlternative # 1\n---")
# Alternative way #1 to handle the previous case
try:
    file3 = None
    file3 = open('file3.txt', 'r') # We are trying to read a nonexistent file
    data = file3.read()
    file3.close()
except IOError:
    print("Some IO error happened")
    if file3:
        print("File exists, closing") # this never gets executed
        file3.close()

print("\nAlternative #2\n---")
# Alternative way #2 to handle the previous case
try:
    file3 = None
    file3 = open('file3.txt', 'r') # We are trying to read a nonexistent file
    data = file3.read()
except IOError:
    print("Some IO error happened")
finally: # Finally block always gets executed, so close file here
    if file3:
        file3.close()

try:
    try:
        file3 = open('file3.txt', 'r') # We are trying to read a nonexistent file
        data = file3.read()
        file3.close()
    except FileNotFoundError:
        print("File not found") # File is not found, so we should do nothing
    except IOError: # file is found but throws an error, so close in this case
        file3.close()
except IOError: # If we fail to close, try again to close
    file3.close()

Write failure case:
---
False

File not found case:
---
File not found

Alternative # 1
---
Some IO error happened

Alternative #2
---
Some IO error happened
File not found


In the above example, we had to handle so many cases of failure where we need to ensure that the file is closed correctly. Trying to recover from a I/O error such as this is very human-error prone, so Python provides constructs to handle this automatically. These are called context managers.

Context management is invoked using the `with` keyword as follows:
```
with open('file.txt', 'r') as file:
    # Do some action with file
    read_data = file.read()
```

After the indented block, the file with automatically close itself since we have no access to the variable outside of the scope of the context manager. One advantage that context managers give is that they automatically clean up in the case of an exception. Thus, we don't have to write cascades of error management code to ensure that we clean up properly. However, we will still need to handle the exceptions according to our program specification and display the correct error messages, when needed.

In [ ]:
# In this example, we will demonstrate writing a string to a file, followed
# by reading it back

STORY = """
A Hare was making fun of the Tortoise one day for being so slow.

"Do you ever get anywhere?" he asked with a mocking laugh.

"Yes," replied the Tortoise, "and I get there sooner than you think. I'll run you a race and prove it."

The Hare was much amused at the idea of running a race with the Tortoise, but for the fun of the thing he agreed. So the Fox, who had consented to act as judge, marked the distance and started the runners off.

The Hare was soon far out of sight, and to make the Tortoise feel very deeply how ridiculous it was for him to try a race with a Hare, he lay down beside the course to take a nap until the Tortoise should catch up.

The Tortoise meanwhile kept going slowly but steadily, and, after a time, passed the place where the Hare was sleeping. But the Hare slept on very peacefully; and when at last he did wake up, the Tortoise was near the goal. The Hare now ran his swiftest, but he could not overtake the Tortoise in time.
"""

with open('hare_and_tortoise.txt', 'w') as hare_tortoise_file:
    hare_tortoise_file.write(STORY)

with open('hare_and_tortoise.txt', 'r') as hare_tortoise_file_read:
    loaded_story = hare_tortoise_file_read.read()

print(loaded_story)


A Hare was making fun of the Tortoise one day for being so slow.

"Do you ever get anywhere?" he asked with a mocking laugh.

"Yes," replied the Tortoise, "and I get there sooner than you think. I'll run you a race and prove it."

The Hare was much amused at the idea of running a race with the Tortoise, but for the fun of the thing he agreed. So the Fox, who had consented to act as judge, marked the distance and started the runners off.

The Hare was soon far out of sight, and to make the Tortoise feel very deeply how ridiculous it was for him to try a race with a Hare, he lay down beside the course to take a nap until the Tortoise should catch up.

The Tortoise meanwhile kept going slowly but steadily, and, after a time, passed the place where the Hare was sleeping. But the Hare slept on very peacefully; and when at last he did wake up, the Tortoise was near the goal. The Hare now ran his swiftest, but he could not overtake the Tortoise in time.



## Data Serialization

Very so often, we want to be able to store structured data into files or serialize different kinds of objects such as lists, dictionaries into file.

For these use cases, it is useful to use the `json` and `csv` libraries of Python. We will not be covering the formats themselves in too much detail, but rather how to use them to read/write data structures.

### JSON

JSON refers to Javascript Object Notation which represents how data is represented in a structured way.

JSON data comprises of a dictionary of key-value pairs:
```
{
    "key1": value1,
    "key2": value2,
    ...
    "keyN": valuen
}
```

For the JSON format, the keys are double quoted strings and can be any string (including empty string!). 

Values can be of the following types:
* Numbers (int or float)
* Strings
* Other JSON dictionaries
* Lists (of any kind of value)

For example, if we want to represent the information about our course, we can do it as follows

```
{
    "id": 10,
    "course_code": "NLP243",
    "term": {
        "year": 2020,
        "quarter": "Fall"
    },
    "name": "Machine Learning for NLP",
    "instructor": {
        "id": 1,
        "name": "Dilek Hakkani-Tur"
    },
    "tas": [
        {
            "id": 2,
            "name": "Rishi Rajasekaran"
        },
        {
            "id": 3,
            "name": "Zekun Zhao"    
        }
    ]
}
```

In the above object, we have "id" keys which are stored as integers, "course_code" and "name" stored as strings, "instructor"  and "term" as other dictionary objects, and "tas" as a list of other objects. 

In Python, we use the `json` library to store data as json. It supports creating JSON strings out of dictionaries or other basic data types and reading/writing them with files.

In [ ]:
import json

# This block demonstrates converting a python dictionary into a JSON string

# using our course object, this is a python dictionary
course = {
    "id": 10,
    "course_code": "NLP243",
    "term": {
        "year": 2020,
        "quarter": "Fall"
    },
    "name": "Machine Learning for NLP",
    "instructor": {
        "id": 1,
        "name": "Dilek Hakkani-Tur"
    },
    "tas": [
        {
            "id": 2,
            "name": "Rishi Rajasekaran"
        },
        {
            "id": 3,
            "name": "Zekun Zhao"    
        }
    ]
}
print("Original course object:")
print(course)
print(type(course))

# Convert the dictionary to a JSON string
course_json = json.dumps(course)
print("\nSeralized course into JSON:")
print(course_json)
print(type(course_json))

course_deserialized = json.loads(course_json)
# Try and spot the difference on how python displays a dict vs the JSON string.
print("\nDeserialized course from JSON:")
print(course_deserialized) 
print(type(course_deserialized))



Original course object:
{'id': 10, 'course_code': 'NLP243', 'term': {'year': 2020, 'quarter': 'Fall'}, 'name': 'Machine Learning for NLP', 'instructor': {'id': 1, 'name': 'Dilek Hakkani-Tur'}, 'tas': [{'id': 2, 'name': 'Rishi Rajasekaran'}, {'id': 3, 'name': 'Zekun Zhao'}]}
<class 'dict'>

Seralized course into JSON:
{"id": 10, "course_code": "NLP243", "term": {"year": 2020, "quarter": "Fall"}, "name": "Machine Learning for NLP", "instructor": {"id": 1, "name": "Dilek Hakkani-Tur"}, "tas": [{"id": 2, "name": "Rishi Rajasekaran"}, {"id": 3, "name": "Zekun Zhao"}]}
<class 'str'>

Deserialized course from JSON:
{'id': 10, 'course_code': 'NLP243', 'term': {'year': 2020, 'quarter': 'Fall'}, 'name': 'Machine Learning for NLP', 'instructor': {'id': 1, 'name': 'Dilek Hakkani-Tur'}, 'tas': [{'id': 2, 'name': 'Rishi Rajasekaran'}, {'id': 3, 'name': 'Zekun Zhao'}]}
<class 'dict'>


In [ ]:
# We also have convencience methods for reading/writing JSON into files
import json

course = {
    "id": 10,
    "course_code": "NLP243",
    "term": {
        "year": 2020,
        "quarter": "Fall"
    },
    "name": "Machine Learning for NLP",
    "instructor": {
        "id": 1,
        "name": "Dilek Hakkani-Tur"
    },
    "tas": [
        {
            "id": 2,
            "name": "Rishi Rajasekaran"
        },
        {
            "id": 3,
            "name": "Zekun Zhao"    
        }
    ]
}

with open('nlp243.json', 'w') as nlp_json_file:
    json.dump(course, nlp_json_file) # Dump into file

with open('nlp243.json', 'r') as nlp_json_file:
    deserialized_courses = json.load(nlp_json_file) # Load from file

# dump and load operate on file objects
# dump: wraps calling dumps and file.write
# load: wraps calling file.read and loads

print("Loaded course information:")
print(deserialized_courses)

Loaded course information:
{'id': 10, 'course_code': 'NLP243', 'term': {'year': 2020, 'quarter': 'Fall'}, 'name': 'Machine Learning for NLP', 'instructor': {'id': 1, 'name': 'Dilek Hakkani-Tur'}, 'tas': [{'id': 2, 'name': 'Rishi Rajasekaran'}, {'id': 3, 'name': 'Zekun Zhao'}]}


### CSV

CSV stands for Comma Separated Values and is used to store tabular data. 

Here is an example of tabular data:

| First Name | Last Name   | Role       |
|------------|-------------|------------|
| Dilek      | Hakkani-Tur | Instructor |
| Rishi      | Rajasekaran | TA         |
| Zekun      | Zhao        | TA         |

This table can be stored easily in a text file as:
```
First Name, Last Name, Role\n
Dilek,Hakkani-Tur,Instructor\n
Rishi,Rajasekaran,TA\n
Zekun,Zhao,TA\n
```

Here, the commas act as column delimiters and the `\n` acts as a row delimiter.

There will be many cases where we will be using CSVs to store data, including the homeworks where we will provide all data files as CSVs.

We can commonly represent tabular data in Python in two ways:
* A list of tuples/lists
* A list of dicts

These can then be serialized easily to a CSV file using the `csv` module.

In [ ]:
import csv

##############################################
# Reading and writing lists of data directly #
##############################################
course_staff = [
                ('Dilek', 'Hakkani-Tur', 'Instructor'),
                ('Rishi', 'Rajasekaran', 'TA'),
                ('Zekun', 'Zhao', 'TA')
]

with open('nlp243_staff.csv', 'w') as staff_csv:
    fieldnames = ['First Name', 'Last Name', 'Role']

    staff_writer = csv.writer(staff_csv, delimiter=',')
    staff_writer.writerow(fieldnames)
    staff_writer.writerows(course_staff)

with open('nlp243_staff.csv', 'r') as staff_csv:

    staff_reader = csv.reader(staff_csv)

    for row in staff_reader:
        print(row)

###########################################
# Reading and writing lists of dictionary #
###########################################

course_staff_dict_list = [
    {'First Name': 'Dilek', 'Last Name': 'Hakkani-Tur', 'Role': 'Instructor'},
    {'First Name': 'Rishi', 'Last Name': 'Rajasekaran', 'Role': 'TA'},
    {'First Name': 'Zekun', 'Last Name': 'Zhao', 'Role': 'TA'}
]

with open('nlp243_staff2.csv', 'w') as staff_csv2:
    writer = csv.DictWriter(staff_csv2, 
                            fieldnames = ['First Name', 'Last Name', 'Role'])
    
    writer.writerows(course_staff_dict_list)

with open('nlp243_staff2.csv', 'r') as staff_csv2:
    reader = csv.DictReader(staff_csv2, 
                            fieldnames = ['First Name', 'Last Name', 'Role'])
    
    for row in reader:
        print(row)

with open('nlp243_staff.csv', 'r') as staff_csv:
    print("Raw CSV:")
    print(staff_csv.read())

['First Name', 'Last Name', 'Role']
['Dilek', 'Hakkani-Tur', 'Instructor']
['Rishi', 'Rajasekaran', 'TA']
['Zekun', 'Zhao', 'TA']
OrderedDict([('First Name', 'Dilek'), ('Last Name', 'Hakkani-Tur'), ('Role', 'Instructor')])
OrderedDict([('First Name', 'Rishi'), ('Last Name', 'Rajasekaran'), ('Role', 'TA')])
OrderedDict([('First Name', 'Zekun'), ('Last Name', 'Zhao'), ('Role', 'TA')])
Raw CSV:
First Name,Last Name,Role
Dilek,Hakkani-Tur,Instructor
Rishi,Rajasekaran,TA
Zekun,Zhao,TA



### Miscellaneous Formats

For the interest of brevity, we have covered only CSV and JSON. There are other data storage formats such as XML and YAML for which libraries exist. The principles we covered for JSON and CSV, however are very similar to the other formats (though not exactly) and will make it easy to pick up new libraries.

## Storing Python Objects - Pickle

The earlier data formats described are convenient to store data. However, there may be a necessity to also store Python objects directly, especially if those objects have specific function implementations.

This will come into picture when you're storing your trained models, since we'd also want to also capture their behavior - for e.g. how their `predict` methods are called.

Pickle is a Python standard that allows us to arbitrarily serialize any Python data - functions, object, classes, etc. It is a binary serialization format - i.e. it is not stored as readable text in a file. 

Therefore, to use pickle to store data, we will need to add a new flag to file open that indicates we're reading/writing byte-oriented data - `b`.

In [ ]:

import pickle as pkl

#####################################
# Example 1 - Serializing functions #
#####################################

def f(x):
    return x*x

# Write our function to file in the form of bytes
with open('dumped_code.pkl', 'wb') as function_pkl_file:
    pkl.dump(f, function_pkl_file)

# Read our function from the file
with open('dumped_code.pkl', 'rb') as function_pkl_file:
    g = pkl.load(function_pkl_file)

print(g(3))

############################################
# Example 2 - Deserializing a TF-IDF model #
############################################

# Setup dependencies
!pip install rank_bm25
import nltk
nltk.download('punkt')
import numpy as np
import string
from pprint import pprint

# This is an example of a Pickle file that I created in the course of my summer
# research work, to create an index of data for retrieving Reddit fun facts
# for a model I was training

# Link to Pickle file:
# https://drive.google.com/file/d/1AdVRtjaCuxvXA7ErCdJgdwTioFoZBufn/view?usp=sharing

# Either download the file or upload it to your Colab runtime to be able to use it
knowledge_index_path = "/content/drive/My Drive/NLP243/Sections/Section 02 - Files/knowledge_index.pkl"


with open(knowledge_index_path, 'rb') as knowledge_index_pickle:
    knowledge_index = pkl.load(knowledge_index_pickle)

# This class takes an index containing a TF-IDF vectorizer,
# sentences of some fun facts and TF-IDF vectors for the fun facts.
# 
# Using this information, it tries to retrieve the top-n closest fun facts
# to the text that was provided.
class TfIdfRankerRetriever(object):
    """
    A module that performs retrieval from an index and also performs top-n ranking.
    This forms a component of the heuristic knowledge selection policy for the KD-PD-NRG
    """

    def _clean(self, s):
        return ''.join([c for c in s.lower() if c not in string.punctuation])

    def __init__(self, knowledge_index, new_index=False):
        if new_index:
            self.tfidf_vec = knowledge_index["vectorizer"]
            self.knowledge_sentences = knowledge_index["knowledge"]
            self.vectorized_sentences = knowledge_index["knowledge_vecs"]
        else:
            self.tfidf_vec = knowledge_index["tfidf_vec"]
            self.knowledge_sentences = knowledge_index["knowledge_list"]
            self.vectorized_sentences = self.tfidf_vec.transform(self.knowledge_sentences)

    def get_top_n(self, query, n=5):
        # These two lines are derived from dynamic.py of the baseline code, with modifications
        # to enable top-n selection
        query_vector = self.tfidf_vec.transform([self._clean(query)])
        similarity = np.squeeze(np.asarray(query_vector.dot(self.vectorized_sentences.transpose()).todense()))
        top_n_indices = similarity.argsort()[-n:][::-1].tolist()
        retrieve_rank_list = [(self.knowledge_sentences[i], similarity[i]) for i in top_n_indices]
        return retrieve_rank_list

ranker = TfIdfRankerRetriever(knowledge_index)

pprint(ranker.get_top_n('I like fish'))

9
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


[('there is a fish, called the Barreleye Fish, that has a transparent head',
  0.3590835641510005),
 ('During the Brisbane G20 summit, when Putin shook the Canadian Prime '
  'Minister\'s hand, the latter said "I guess I\'ll shake your hand, but I '
  'have only one thing to say to you. You need to get out of Ukraine."',
  0.24303583493681213),
 ('a Seahorse is the only fish to have a neck', 0.24205519306373924),
 ('There is a deep-water fish called the Black Swallower (or great swallower) '
  'that can swallow whole fish twice its length and up to ten times it mass.',
  0.2252799662732104),
 ('freshwater fish only "drink" water through their skin via osmosis, while '
  'saltwater fish also drink water through their mouths',
  0.21354726243656907)]


In [ ]:
### Appendix

def add(a, b):
    return a + b

def addOneToFn(f):
    return lambda a, b: f(a, b) + 1

addWithOne = addOneToFn(add)
print(addWithOne(1, 2))

def multiply(a, b):
    return a * b

multiplyPlusOne = addOneToFn(multiply)

print(multiplyPlusOne(2, 3))

def naturalNumbers():
    i = 1
    while True:
        yield i
        i = i + 1

for i, val in enumerate(naturalNumbers()):
    if i < 100:
        print(val)
    else:
        break

4
7
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
